## Setup

In [ ]:
# Install packages and frameworks

import tensorflow as tf
import os
import visualkeras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow import keras
from keras.utils import plot_model
from keras.callbacks import CSVLogger

# expecting 2.11
# if 2.7, than logging errors will show "Cleanup called..."
print(tf.__version__)

In [ ]:
# manually set a filename to store and compare reports
os.environ['FILENAME'] = 'experiment'

In [ ]:
# scratch directory is apart of the .gitignore to ensure it is not committed to git
%env SCRATCH=../scratch
! [ -e "${SCRATCH}" ] || mkdir -p "${SCRATCH}"

scratch_path = os.environ.get('SCRATCH', './scratch')

# Load the saved datasets

The TFRecord format is a simple format for storing a sequence of binary records.

## Load train

In [ ]:
path = scratch_path + '/tf_datasets/train/'
train_ds = tf.data.Dataset.load(path)
train_ds.element_spec

## Load validate

In [ ]:
path = scratch_path + '/tf_datasets/validate/'
validation_ds = tf.data.Dataset.load(path)
validation_ds.element_spec

## Load test

In [ ]:
path = scratch_path + '/tf_datasets/test/'
test_ds = tf.data.Dataset.load(path)
test_ds.element_spec

## Configure the datasets for performance

Let's make sure to use buffered prefetching so we can yield data from disk without having I/O become blocking. These are two important methods you should use when loading data.

1. `Caching` a dataset, either in memory or on local storage. This will save some operations (like file opening and data reading) from being executed during each epoch.
1. `Prefetching` overlaps the preprocessing and model execution of a training step. While the model is executing training step s, the input pipeline is reading the data for step s+1. Doing so reduces the step time to the maximum (as opposed to the sum) of the training and the time it takes to extract the data.

[Optimising your input pipeline performance with tf.data (part 1)](https://towardsdatascience.com/optimising-your-input-pipeline-performance-with-tf-data-part-1-32e52a30cac4)

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
validation_ds = validation_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = validation_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Check the device spec

In [ ]:
# display physical devices
tf.config.list_physical_devices()

# Create the model

tf.keras models are optimized to make predictions on a batch, or collection, of examples at once. Accordingly, even though you're using a single image, you need to add it to a list:

## Setup the layers

The Sequential model consists of three convolution blocks (tf.keras.layers.Conv2D) with a max pooling layer (tf.keras.layers.MaxPooling2D) in each of them. There's a fully-connected layer (tf.keras.layers.Dense) with 128 units on top of it that is activated by a ReLU activation function ('relu'). This model has not been tuned in any way—the goal is to show you the mechanics using the datasets you just created. To learn more about image classification, visit the Image classification tutorial.

Here's an example of Python code using Keras to create a sequential classification model that accepts 96x96x1 input images.

In this example, we start with three convolutional layers, each followed by a max pooling layer to downsample the input. Then, we flatten the output from the convolutional layers and add two fully connected layers. Finally, we add an output layer with the number of classes and compile the model using an optimizer (e.g., Adam) and a loss function (e.g., categorical cross-entropy).

```
import tensorflow as tf    

model = tf.keras.Model(...)

# Run training on GPU
with tf.device('/gpu:0'):
    model.fit(...)

# Run inference on CPU
with tf.device('/cpu:0'):
    model.predict(...)
    
```

## Set some variables

In [ ]:
# set variables for consistency
img_height = 96              # desired height
img_width = 96               # desired width
batch_size = 32              # batch inputs in 32
seed_train_validation = 42   # Must be same for train_ds and val_ds
validation_split = 0.3       # move 30% of the data into validation
class_names = ['left', 'right']

dataFormat="channels_last"
num_classes = len(class_names)
inputShape=(img_height, img_width, 1)
chanDim = -1

## Set up the layers
The basic building block of a neural network is the layer. Layers extract representations from the data fed into them. Hopefully, these representations are meaningful for the problem at hand.

Most of deep learning consists of chaining together simple layers. Most layers, such as tf.keras.layers.Dense, have parameters that are learned during training.


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=inputShape),
    tf.keras.layers.Reshape(target_shape=inputShape),

    # layer 1
    tf.keras.layers.Conv2D(96, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    # layer 2
    tf.keras.layers.Conv2D(96, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    # layer 3
    tf.keras.layers.Conv2D(96, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    # layer 4
    #tf.keras.layers.Conv2D(96, 3, activation='relu'),
    #tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    # layer 5
    #tf.keras.layers.Conv2D(96, 3, activation='relu'),
    #tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(num_classes),
    tf.keras.layers.Softmax()
])

## Compile the model

Before the model is ready for training, it needs a few more settings. These are added during the model's compile step:

Loss function —This measures how accurate the model is during training. You want to minimize this function to "steer" the model in the right direction.
Optimizer —This is how the model is updated based on the data it sees and its loss function.
Metrics —Used to monitor the training and testing steps. The following example uses accuracy, the fraction of the images that are correctly classified.

In [ ]:
model.compile(optimizer='adam',
            loss=tf.keras.losses.CategoricalCrossentropy(
                from_logits=False,
                label_smoothing=0.0,
                axis=-1,
                #reduction=losses_utils.ReductionV2.AUTO,
                name='categorical_crossentropy'),
            metrics=['accuracy'])

In [ ]:
model.summary()

### Visualize the model architecture

In [ ]:
# Plot the resulting model architecture: https://www.tensorflow.org/api_docs/python/tf/keras/utils/plot_model
filename = os.getenv('FILENAME')

to_file = scratch_path + '/reports/' + filename + '-model_plot.png'
tf.keras.utils.plot_model(model, show_shapes=True,to_file=to_file)

In [ ]:
# Visualize the model
to_file = scratch_path + '/reports/' + filename + '-model_visual.png'
visualkeras.layered_view(model, legend=True, draw_volume=True, to_file=to_file)

# Train the model
Training the neural network model requires the following steps:

Feed the training data to the model. In this example, the training data is in the train_images and train_labels arrays.
The model learns to associate images and labels.
You ask the model to make predictions about a test set—in this example, the test_images array.
Verify that the predictions match the labels from the test_labels array.

In [ ]:
filename = scratch_path + '/reports/' + filename + '-score.csv'
csv_logger = CSVLogger(filename, append=True, separator=';')
model.fit(train_ds, epochs=10, validation_data=validation_ds, callbacks=[csv_logger])

# Evaluate the model

Next, compare how the model performs on the test dataset:

In [ ]:
test_loss, test_acc = model.evaluate(test_ds, verbose=2)

# Score the model

In [ ]:
# Generate generalization metrics
score = model.evaluate(test_ds, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')